# IMDB Movie Review Dataset

This is a library to download and parse the [IMDB's Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/) dataset and a demo of a transformer based model. The dataset has 25K training, and 25K test dataset, plus 50K unlabeled examples.

It's inspired on [Keras' Text classification with Transformer](https://keras.io/examples/nlp/text_classification_with_transformer/) demo.




In [1]:
!*go mod edit -replace gomlx=${HOME}/Projects/gomlx

## Data Preparation

### Downloading data files

To download, uncompress and untar to the local directory, simply do the following. Notice if it's already downloaded in the given `--data` directory, it returns immediately.

In [2]:
import (
    "github.com/gomlx/gomlx/examples/imdb"
    "github.com/gomlx/gomlx/ml/data"
)

var flagDataDir = flag.String("data", "~/tmp/imdb", "Directory to cache downloaded and generated dataset files.")

func AssertNoError(err error) {
    if err != nil {
        log.Fatalf("Failed: %+v", err)
    }
}

func AssertDownloaded() {
    *flagDataDir = data.ReplaceTildeInDir(*flagDataDir)
    if !data.FileExists(*flagDataDir) {
        AssertNoError(os.MkdirAll(*flagDataDir, 0777))
    }

    AssertNoError(imdb.Download(*flagDataDir))
}

%%
AssertDownloaded()

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.


### Sampling some examples

It creates a small dataset and print out some random examples.

It also defines the `DType`, used for all internal representations of the model, and the flag `--max_len` that defines the maximum number of tokens used per observation. This will beused in the modeling later.

In [3]:
import (
    "github.com/gomlx/gomlx/examples/imdb"
)

// DType used for the models.
const DType = shapes.Float32

var (
    flagMaxLen              = flag.Int("max_len", 200, "Maximum number of tokens to take from observation.")
)

func Sample() {
    ds := imdb.NewDataset("Test", imdb.Test, *flagMaxLen, 3, DType, true, nil)
    _, inputs, labels, err := ds.Yield()
    AssertNoError(err)
    labelsData := shapes.CastAsDType(labels[0].Local().Data(), shapes.Int64).([]int)
    for ii := 0; ii < 3; ii++ {
        fmt.Printf("\nlabel=%v, input=%q\n", labelsData[ii], imdb.InputToString(inputs[0], ii))
    }
    fmt.Println()
}

%% --max_len=200
AssertDownloaded()
Sample()


> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.

label=0, input="that is the case it could have been done better much better this film does not know what it wants to be a drama a comedy a teenage wasteland film or a gamut of other things i say this not as a single genre oriented person but as a person who loves multi genre pieces such as those mastered by stanley kubrick the reason i feel it does not work for garden state is because rather than blending the genres together it jumps around one scene is one genre the next another and back again so on i have heard many people tell me to cut zach braff some slack after all it was his first feature film and he debuted as director writer and lead actor impressive as it was a first film i should think that with as much money as he has he could do better there have been much better first time feature film directors michel gondry human natu

## Training

We will create 3 different types of models for this demo: **Bag of Words** (or simply **bow**), **CNNs** and **Transformers**.

### Model Support

We first define here a few componens shared among all models:

* `Normalize` according to flag, works for sequence nodes (shaped `[batch_size, sequence_len, embedding_dim]`) and feature nodes (shaped `[batch_size, embedding_dim]`).
* `EmbedTokensGraph` that embeds the tokens, before being consumed by the models.
* `ReadoutGraph` that takes the embeddings after they are pooled on the sequence axis (so shaped `[batch_size, embedding_dim]` it adds a FNN (feed-forward neural network) with a few layers and convert to the final logits.

We also define the corresponding hyperparameters.

In [6]:
import (
    . "github.com/gomlx/gomlx/graph"
    
    "github.com/gomlx/gomlx/ml/context"
    "github.com/gomlx/gomlx/types/shapes"
)

var (
    flagNormalization    = flag.String("norm", "layer", "Type of normalization to use. Valid values are \"none\", \"batch\", \"layer\".")
    flagDropoutRate      = flag.Float64("dropout", 0.15, "Dropout rate")
    flagMaxVocab            = flag.Int("max_vocab", 20000, "Top most frequent words to consider, the rest is considered unknown.")
    flagTokenEmbeddingSize  = flag.Int("token_embed", 32, "Size of token embedding table. There are ~140K unique tokens")
    flagNumHiddenLayers     = flag.Int("hidden_layers", 2, "Number of output hidden layers, stacked with residual connection.")
    flagNumNodes            = flag.Int("num_nodes", 32, "Number of nodes in output hidden layers.")
    flagWordDropoutRate  = flag.Float64("word_dropout", 0, "Dropout rate for whole words of the input")
)

// Normalize x according to --norm flag. Works for sequence nodes (rank-3) or plain feature nodes (rank-2).
func Normalize(ctx *context.Context, x *Node) *Node {
    switch *flagNormalization {
    case "layer":
        if x.Rank() == 3 {
            // Normalize sequence.
            return layers.LayerNormalization(ctx, x, -2, -1).
                LearnedOffset(true).LearnedScale(true).ScaleNormalization(true).Done()
        } else {
            // Normalize features only.
            return layers.LayerNormalization(ctx, x, -1).Done()
        }
    case "batch":
        return layers.BatchNormalization(ctx, x, -1).Done()
    case "none":
        return x
    }
    g := x.Graph()
    g.SetErrorf("invalid normalization selected %q -- valid values are batch, layer, none", *flagNormalization)
    return g.InvalidNode()
}

// EmbedTokensGraph creates embeddings for tokens and returns them along with the mask of used tokens --
// set to false where padding was used.
func EmbedTokensGraph(ctx *context.Context, tokens *Node) (embed, mask *Node) {
    g := tokens.Graph()
    mask = NotEqual(tokens, ZerosLike(tokens)) // Mask of tokens actually used.

    // The token ids are indexed by frequency. Truncate to the vocabulary size considered, replacing
    // ids higher than that by 0.
    maxVocab := len(imdb.LoadedVocab.ListEntries)
    if maxVocab > *flagMaxVocab {
        maxVocab = *flagMaxVocab
    }

    // Limits tokens to the maxVocab.
    tokens = Where(GreaterOrEqual(tokens, Const(g, maxVocab)),
        MulScalar(OnesLike(tokens), float64(maxVocab-1)),
        tokens)

    // Embed tokens: shape=[batchSize, maxLen, embedDim]
    embed = layers.Embedding(ctx.In("tokens"), tokens, DType, maxVocab, *flagTokenEmbeddingSize)
    embed = Where(mask, embed, ZerosLike(embed))
    
    if *flagWordDropoutRate > 0 {
        dims := embed.Shape().Dimensions[:len(embed.Shape().Dimensions)-1]
        dropoutMask := Ones(g, shapes.Make(DType, dims...))
        dropoutMask = layers.Dropout(ctx, dropoutMask, ConstAsDType(g, DType, *flagWordDropoutRate))
        dropoutMask = ExpandDims(dropoutMask, -1)
        embed = Mul(embed, dropoutMask)
    }    
    return
}

// Activation function used for models.
func Activation(x *Node) *Node {
    return layers.Relu(x)
}

// ReadoutGraph takes the embeddings after they have been pooled on the sequence axis, so shaped `[batch_size, embed_dim]`
// adds a FNN on top and readout the final logit.
func ReadoutGraph(ctx *context.Context, embed *Node) *Node {
    g := embed.Graph()
    if !g.Ok() {
        return g.InvalidNode()
    }
    var dropoutRate *Node
    if *flagDropoutRate > 0 {
        dropoutRate = ConstAsDType(g, DType, *flagDropoutRate)
    }

    // Output layers.
    for ii := 0; ii < *flagNumHiddenLayers; ii++ {
        ctx := ctx.In(fmt.Sprintf("output_dense_%d", ii))
        residual := embed
        if *flagDropoutRate > 0 {
            embed = layers.Dropout(ctx, embed, dropoutRate)
        }
        //embed = Tanh(embed)
        embed = Activation(embed)
        embed = layers.DenseWithBias(ctx, embed, *flagNumNodes)
        embed = Normalize(ctx, embed)
        if ii > 0 {
            // Add residual connection.
            embed = Add(embed, residual)
        }
    }

    // Final embed layer with dimension 1.
    {
        ctx := ctx.In("readout")
        if *flagDropoutRate > 0 {
            embed = layers.Dropout(ctx, embed, dropoutRate)
        }
        embed = Activation(embed)
        embed = layers.DenseWithBias(ctx, embed, 1)
    }
    return embed
}

### Bag Of Words Model

This is the simples model we are going to train. It's basically a no-op that connects the embedding table we defined above with the `ReadoutGraph` FNN afterwards.

We define here placeholders for our future *CNN* and *Transformer* models.

Finally we test that the shape is correct. To actually train and evaluate we still need to define the training loop, what we do in the following section.

In [7]:
import (
    . "github.com/gomlx/gomlx/graph"
    
    "github.com/gomlx/gomlx/ml/context"
)

var (
    // ML Manager creation flags.
    flagNumThreads  = flag.Int("num_threads", -1, "Number of threads. Leave as -1 to use as many as there are cores.")
    flagNumReplicas = flag.Int("num_replicas", 1, "Number of replicas.")
    flagPlatform    = flag.String("platform", "", "Platform to use, if empty uses the default one.")

    // Model type.
    flagModel            = flag.String("model", "transformer", "Model type: bow or transformer.")
)

// ModelGraph builds the model for our demo. It returns the logits, not the predictions, which works with most losses.
func ModelGraph(ctx *context.Context, spec any, inputs []*Node) []*Node {
	_ = spec // Not used.
	tokens := inputs[0]
	g := tokens.Graph()
	if !g.Ok() {
		return nil
	}
	embed, mask := EmbedTokensGraph(ctx, tokens)

	// Normalization function.
	if *flagModel == "bow" {
		// Bag-Of-Words model doesn't do anything, it's just the embedding table for each token.
	} else if *flagModel == "cnn" {
		embed = Conv1DGraph(ctx, embed, mask)
	} else if *flagModel == "transformer" {
		embed = TransformerGraph(ctx, tokens, embed, mask)
	} else {
		g.SetErrorf("unknown model type %q, only types \"bow\", \"cnn\" and \"transformer\" are implemented", *flagModel)
		return nil
	}

	// Sum-up per-token embeddings and do a FNN on the output. From now on, the dimensions are `[batch_dim, embed_dim]`
	// Notice we are not using mask.
	embed = ReduceMax(embed, 1)
	logits := ReadoutGraph(ctx, embed)
	return []*Node{logits}
}

func Conv1DGraph(ctx *context.Context, embed, mask *Node) *Node {
    log.Panic("Not implemented.")
    return nil
}

func TransformerGraph(ctx *context.Context, tokens, embed, mask *Node) *Node {
    log.Panic("Not implemented.")
    return nil
}

%% --model=bow --platform=Host

// Let's test that the logits are coming out with the right shape: we want [batch_size, 10], since there are 10 classes.
AssertDownloaded()
manager := BuildManager().NumThreads(*flagNumThreads).NumReplicas(*flagNumReplicas).Platform(*flagPlatform).MustDone()
ds := imdb.NewDataset("Test", imdb.Test, *flagMaxLen, 3, DType, true, nil)
_, inputs, _, err := ds.Yield()
AssertNoError(err)

g := manager.NewGraph("test")
ctx := context.NewContext(manager)
logits := ModelGraph(ctx, nil, []*Node{Const(g, inputs[0])})
AssertNoError(g.Error())
AssertNoError(ctx.Error())
fmt.Printf("Logits shape for batch_size=%d: %s\n", 3, logits[0].Shape())

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Logits shape for batch_size=3: (Float32)[3 1]


### Training Loop

Training loop, with the usual bells and whistles. We also do a quick test with 100 steps to check things are working.

In [8]:
import (
    . "github.com/gomlx/gomlx/graph"
    
    "github.com/gomlx/gomlx/ml/context"
    "github.com/gomlx/gomlx/ml/train"
)

var (
    flagOptimizer        = flag.String("optimizer", "adam", "Optimizer, options: adam or sgd.")
    flagNumSteps         = flag.Int("steps", 5000, "Number of gradient descent steps to perform")
    flagBatchSize        = flag.Int("batch", 32, "Batch size for training")
    flagLearningRate     = flag.Float64("learning_rate", 0.0001, "Initial learning rate.")
    flagL2Regularization = flag.Float64("l2_reg", 0, "L2 regularization on kernels. It doesn't interact well with --batch_norm or with --optimizer=adam.")
    
    flagCheckpoint       = flag.String("checkpoint", "", "Directory save and load checkpoints from. If left empty, no checkpoints are created.")
    flagCheckpointKeep   = flag.Int("checkpoint_keep", 10, "Number of checkpoints to keep, if --checkpoint is set.")

    // Training UI:
    flagUseProgressBar = flag.Bool("bar", true, "If to display a progress bar during training")
    flagNumPlotPoints  = flag.Int("plot_points", 50, "Number points to plot.")
)

func trainModel() {
    // Make sure data is downloaded.
    AssertDownloaded()

    // Manager handles creation of ML computation graphs, accelerator resources, etc.
    manager := BuildManager().NumThreads(*flagNumThreads).NumReplicas(*flagNumReplicas).Platform(*flagPlatform).MustDone()
    fmt.Printf("Platform: %s\n", manager.Platform())

    // Datasets.
    var trainDS, trainEvalDS, testEvalDS train.Dataset
    trainDS = imdb.NewDataset("train", imdb.Train, *flagMaxLen, *flagBatchSize, DType, true, nil)
    trainEvalDS = imdb.NewDataset("train-eval", imdb.Train, *flagMaxLen, *flagBatchSize, DType, false, nil)
    testEvalDS = imdb.NewDataset("test-eval", imdb.Test, *flagMaxLen, *flagBatchSize, DType, false, nil)

    // Parallelize generation of batches, to prevent the dataset from being a bottleneck to the acceleartor (GPU/TPU).
    trainDS = data.NewParallelDataset(trainDS, 0, 0)
    trainEvalDS = data.NewParallelDataset(trainEvalDS, 0, 0)
    testEvalDS = data.NewParallelDataset(testEvalDS, 0, 0)

    // Metrics we are interested.
    meanAccuracyMetric := metrics.NewMeanBinaryLogitsAccuracy("Mean Accuracy", "#acc")
    movingAccuracyMetric := metrics.NewMovingAverageBinaryLogitsAccuracy("Moving Average Accuracy", "~acc", 0.01)

    // Context holds the variables and hyperparameters for the model.
    ctx := context.NewContext(manager)
    ctx.SetParam(optimizers.LearningRateKey, *flagLearningRate)
    ctx.SetParam(layers.L2RegularizationKey, *flagL2Regularization)

    // Checkpoints saving.
    var checkpoint *checkpoints.Handler
    if *flagCheckpoint != "" {
        checkpointPath := data.ReplaceTildeInDir(*flagCheckpoint)
        if !path.IsAbs(checkpointPath) {
            checkpointPath = path.Join(*flagDataDir, checkpointPath)
        }
        var err error
        checkpoint, err = checkpoints.Build(ctx).Dir(checkpointPath).Keep(*flagCheckpointKeep).Done()
        AssertNoError(err)
    }

    // Create a train.Trainer: this object will orchestrate running the model, feeding
    // results to the optimizer, evaluating the metrics, etc. (all happens in trainer.TrainStep)
    loss := losses.BinaryCrossentropyLogits
    trainer := train.NewTrainer(
        manager, ctx, ModelGraph, loss,
        optimizers.MustOptimizerByName(*flagOptimizer),
        []metrics.Interface{movingAccuracyMetric}, // trainMetrics
        []metrics.Interface{meanAccuracyMetric})   // evalMetrics

    // Use standard training loop.
    loop := train.NewLoop(trainer)
    if *flagUseProgressBar {
        commandline.AttachProgressBar(loop) // Attaches a progress bar to the loop.
    }

    if checkpoint != nil {
        train.NTimesDuringLoop(loop, 2*(*flagCheckpointKeep), "checkpointing", 100, func(_ *train.Loop, _ []tensor.Tensor) error {
            return checkpoint.Save()
        })
    }

    // Attach a margaid plots.
    if *flagNumPlotPoints > 0 {
        margaid.New(1024, 400, trainEvalDS, testEvalDS).DynamicUpdates().Attach(loop, *flagNumPlotPoints)
    }

    // Loop for given number of steps.
    _, err := loop.RunSteps(trainDS, *flagNumSteps)
    AssertNoError(err)

    // Report final evaluation.
    fmt.Println()
    err = commandline.ReportEval(trainer, trainEvalDS, testEvalDS)
    AssertNoError(err)
}

%% --platform=Host --model=bow --steps=100 --plot_points=0
trainModel()


> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Platform: Host
Training (100 steps):  100% [========================================] (141 steps/s) [loss=0.674] [~loss=0.685] [~acc=57.16%]        

Results on train-eval:
	Mean Loss (#loss): 0.702
	Mean Accuracy (#acc): 49.99%
Results on test-eval:
	Mean Loss (#loss): 0.702
	Mean Accuracy (#acc): 50.00%


### Training Bag-Of-Words ("bow") Model

A proper run of our *bow* model.

In [9]:
%% --model=bow --steps=20000 --optimizer=adamw
trainModel()

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Platform: CUDA


Training (20000 steps):  100% [========================================] (701 steps/s) [loss=0.157] [~loss=0.231] [~acc=91.06%]        


<path d="M1.768000e+01,5.222363e+00 L3.461121e+01,5.222363e+00 L5.158486e+01,5.334248e+00 L6.855851e+01,1.040019e+02 L8.553216e+01,1.598911e+02 L1.025058e+02,1.570944e+02 L1.194795e+02,1.595332e+02 L1.364531e+02,1.698698e+02 L1.534268e+02,1.752842e+02 L1.704004e+02,1.749262e+02 L1.873741e+02,1.823766e+02 L2.043477e+02,1.853076e+02 L2.213214e+02,1.858222e+02 L2.382950e+02,1.829360e+02 L2.552686e+02,1.893348e+02 L2.722423e+02,1.907443e+02 L2.892159e+02,1.909905e+02 L3.061896e+02,1.928027e+02 L3.231632e+02,1.867618e+02 L3.401369e+02,1.966510e+02 L3.571105e+02,1.953086e+02 L3.740842e+02,1.923552e+02 L3.910578e+02,1.981053e+02 L4.080315e+02,1.991792e+02 L4.250051e+02,1.921763e+02 L4.419788e+02,1.934963e+02 L4.589524e+02,1.954876e+02 L4.759261e+02,1.979934e+02 L4.928997e+02,1.986646e+02 L5.098734e+02,1.983066e+02 L5.268470e+02,1.962706e+02 L5.438207e+02,2.020206e+02 L5.607943e+02,1.997385e+02 L5.777680e+02,1.991345e+02 L5.947416e+02,2.016403e+02 L6.117153e+02,1.943241e+02 L6.286889e+02,1.926461e+02 L6.456626e+02,2.008796e+02 L6.626362e+02,2.006559e+02 L6.796099e+02,1.993582e+02 L6.965835e+02,1.993135e+02 L7.135572e+02,1.993135e+02 L7.305308e+02,2.015732e+02 L7.475045e+02,2.026471e+02 L7.644781e+02,2.006111e+02 L7.814518e+02,2.007677e+02 L7.984254e+02,2.018417e+02 L8.153991e+02,1.957337e+02 L8.323727e+02,1.986423e+02 L8.493464e+02,2.002755e+02 L8.663200e+02,1.998728e+02 " vector-effect="non-scaling-stroke"/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.200000e+00 L3.461121e+01,5.200000e+00 L5.158486e+01,5.468462e+00 L6.855851e+01,1.088122e+02 L8.553216e+01,1.683036e+02 L1.025058e+02,1.632696e+02 L1.194795e+02,1.672521e+02 L1.364531e+02,1.814817e+02 L1.534268e+02,1.893572e+02 L1.704004e+02,1.902745e+02 L1.873741e+02,1.995819e+02 L2.043477e+02,2.044594e+02 L2.213214e+02,2.046831e+02 L2.382950e+02,2.030275e+02 L2.552686e+02,2.117979e+02 L2.722423e+02,2.128942e+02 L2.892159e+02,2.142814e+02 L3.061896e+02,2.172347e+02 L3.231632e+02,2.115742e+02 L3.401369e+02,2.232085e+02 L3.571105e+02,2.224030e+02 L3.740842e+02,2.204118e+02 L3.910578e+02,2.272133e+02 L4.080315e+02,2.297639e+02 L4.250051e+02,2.224254e+02 L4.419788e+02,2.244166e+02 L4.589524e+02,2.271462e+02 L4.759261e+02,2.311735e+02 L4.928997e+02,2.333661e+02 L5.098734e+02,2.335674e+02 L5.268470e+02,2.321132e+02 L5.438207e+02,2.401005e+02 L5.607943e+02,2.383330e+02 L5.777680e+02,2.393622e+02 L5.947416e+02,2.424274e+02 L6.117153e+02,2.340149e+02 L6.286889e+02,2.325607e+02 L6.456626e+02,2.435685e+02 L6.626362e+02,2.438593e+02 L6.796099e+02,2.429867e+02 L6.965835e+02,2.456045e+02 L7.135572e+02,2.454702e+02 L7.305308e+02,2.488710e+02 L7.475045e+02,2.513097e+02 L7.644781e+02,2.490947e+02 L7.814518e+02,2.514663e+02 L7.984254e+02,2.542630e+02 L8.153991e+02,2.456940e+02 L8.323727e+02,2.508399e+02 L8.493464e+02,2.548000e+02 L8.663200e+02,2.544644e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,4.016994e+01 L3.461121e+01,1.412180e+01 L5.158486e+01,1.806404e+01 L6.855851e+01,4.032986e+01 L8.553216e+01,1.037123e+02 L1.025058e+02,1.267975e+02 L1.194795e+02,1.426188e+02 L1.364531e+02,1.406405e+02 L1.534268e+02,1.498781e+02 L1.704004e+02,1.601274e+02 L1.873741e+02,1.675366e+02 L2.043477e+02,1.669375e+02 L2.213214e+02,1.711165e+02 L2.382950e+02,1.747295e+02 L2.552686e+02,1.798200e+02 L2.722423e+02,1.845764e+02 L2.892159e+02,1.791738e+02 L3.061896e+02,1.865675e+02 L3.231632e+02,1.892510e+02 L3.401369e+02,1.916215e+02 L3.571105e+02,1.957265e+02 L3.740842e+02,1.987298e+02 L3.910578e+02,1.975202e+02 L4.080315e+02,2.007573e+02 L4.250051e+02,2.026894e+02 L4.419788e+02,2.041735e+02 L4.589524e+02,2.070303e+02 L4.759261e+02,2.072989e+02 L4.928997e+02,2.070914e+02 L5.098734e+02,2.084090e+02 L5.268470e+02,2.122601e+02 L5.438207e+02,2.114010e+02 L5.607943e+02,2.149750e+02 L5.777680e+02,2.116493e+02 L5.947416e+02,2.145258e+02 L6.117153e+02,2.113922e+02 L6.286889e+02,2.182973e+02 L6.456626e+02,2.190615e+02 L6.626362e+02,2.196856e+02 L6.796099e+02,2.202338e+02 L6.965835e+02,2.218646e+02


Results on train-eval:


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.533708e+02 L3.461121e+01,2.548000e+02 L5.158486e+01,2.504302e+02 L6.855851e+01,2.392624e+02 L8.553216e+01,1.706577e+02 L1.025058e+02,1.604292e+02 L1.194795e+02,1.538591e+02 L1.364531e+02,1.411338e+02 L1.534268e+02,1.332840e+02 L1.704004e+02,1.313557e+02 L1.873741e+02,1.229594e+02 L2.043477e+02,1.189157e+02 L2.213214e+02,1.175806e+02 L2.382950e+02,1.204254e+02 L2.552686e+02,1.127202e+02 L2.722423e+02,1.115007e+02 L2.892159e+02,1.103799e+02 L3.061896e+02,1.077605e+02 L3.231632e+02,1.155808e+02 L3.401369e+02,1.026467e+02 L3.571105e+02,1.046702e+02 L3.740842e+02,1.091341e+02 L3.910578e+02,1.007152e+02 L4.080315e+02,9.900802e+01 L4.250051e+02,1.087975e+02 L4.419788e+02,1.073079e+02 L4.589524e+02,1.050386e+02 L4.759261e+02,1.009785e+02 L4.928997e+02,9.925271e+01 L5.098734e+02,9.966842e+01 L5.268470e+02,1.053659e+02 L5.438207e+02,9.648987e+01 L5.607943e+02,9.867854e+01 L5.777680e+02,1.002412e+02 L5.947416e+02,9.623806e+01 L6.117153e+02,1.062985e+02 L6.286889e+02,1.115959e+02 L6.456626e+02,9.764226e+01 L6.626362e+02,9.790117e+01 L6.796099e+02,9.974611e+01 L6.965835e+02,1.001680e+02 L7.135572e+02,1.008847e+02 L7.305308e+02,9.962736e+01 L7.475045e+02,9.600494e+01 L7.644781e+02,9.974344e+01 L7.814518e+02,9.700590e+01 L7.984254e+02,9.787213e+01 L8.153991e+02,1.076990e+02 L8.323727e+02,1.091760e+02 L8.493464e+02,1.011230e+02 L8.663200e+02,1.046420e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.533588e+02 L3.461121e+01,2.547230e+02 L5.158486e+01,2.501910e+02 L6.855851e+01,2.381337e+02 L8.553216e+01,1.646437e+02 L1.025058e+02,1.527594e+02 L1.194795e+02,1.434807e+02 L1.364531e+02,1.282210e+02 L1.534268e+02,1.167705e+02 L1.704004e+02,1.131783e+02 L1.873741e+02,1.015165e+02 L2.043477e+02,9.531974e+01 L2.213214e+02,9.291846e+01 L2.382950e+02,9.385593e+01 L2.552686e+02,8.262005e+01 L2.722423e+02,8.151630e+01 L2.892159e+02,7.942152e+01 L3.061896e+02,7.321379e+01 L3.231632e+02,7.912593e+01 L3.401369e+02,6.387751e+01 L3.571105e+02,6.397841e+01 L3.740842e+02,6.547319e+01 L3.910578e+02,5.631191e+01 L4.080315e+02,5.287983e+01 L4.250051e+02,6.128636e+01 L4.419788e+02,5.867134e+01 L4.589524e+02,5.508710e+01 L4.759261e+02,4.737149e+01 L4.928997e+02,4.483194e+01 L5.098734e+02,4.406999e+01 L5.268470e+02,4.657029e+01 L5.438207e+02,3.361724e+01 L5.607943e+02,3.570610e+01 L5.777680e+02,3.482903e+01 L5.947416e+02,2.926303e+01 L6.117153e+02,4.018457e+01 L6.286889e+02,4.045108e+01 L6.456626e+02,2.528090e+01 L6.626362e+02,2.570575e+01 L6.796099e+02,2.549952e+01 L6.965835e+02,2.229596e+01 L7.135572e+02,2.138829e+01 L7.305308e+02,1.655666e+01 L7.475045e+02,1.425212e+01 L7.644781e+02,1.570460e+01 L7.814518e+02,1.501638e+01 L7.984254e+02,8.125490e+00 L8.153991e+02,1.956125e+01 L8.323727e+02,1.150194e+01 L8.493464e+02,6.990772e+00 L8.663200e+02,5.200000e+00 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.537503e+02 L3.461121e+01,2.473296e+02 L5.158486e+01,2.525123e+02 L6.855851e+01,2.496131e+02 L8.553216e+01,1.622988e+02 L1.025058e+02,2.163431e+02 L1.194795e+02,1.655028e+02 L1.364531e+02,2.054395e+02 L1.534268e+02,1.558318e+02 L1.704004e+02,1.012990e+02 L1.873741e+02,8.924353e+01 L2.043477e+02,8.759756e+01 L2.213214e+02,1.095173e+02 L2.382950e+02,1.544389e+02 L2.552686e+02,1.046849e+02 L2.722423e+02,7.070779e+01 L2.892159e+02,1.930257e+02 L3.061896e+02,1.082868e+02 L3.231632e+02,5.602326e+01 L3.401369e+02,9.217462e+01 L3.571105e+02,4.205497e+01 L3.740842e+02,4.766723e+01 L3.910578e+02,8.070806e+01 L4.080315e+02,1.189528e+02 L4.250051e+02,1.054223e+02 L4.419788e+02,6.636345e+01 L4.589524e+02,1.339921e+02 L4.759261e+02,5.496560e+01 L4.928997e+02,8.449727e+01 L5.098734e+02,1.282648e+02 L5.268470e+02,6.263175e+01 L5.438207e+02,1.292627e+02 L5.607943e+02,1.254287e+02 L5.777680e+02,3.773471e+01 L5.947416e+02,3.027861e+01 L6.117153e+02,9.861816e+01 L6.286889e+02,8.379324e+01 L6.456626e+02,1.147264e+02 L6.626362e+02,7.649096e+01 L6.796099e+02,7.338602e+00 L6.965835e+02,7.607846e+01

	Mean Loss (#loss): 0.149
	Mean Accuracy (#acc): 94.57%
Results on test-eval:
	Mean Loss (#loss): 0.368
	Mean Accuracy (#acc): 84.81%


### CNN Model

Below we redefine our CNN model function, with 2 convolution layers, and train on it.

Notice how well it can overfit to the training data ... but it doesn't help the test results. To improve this one needs some careful regularization.

In [10]:
// Conv1DGraph stacks two 1D convolution layers on top of the sequence embeddings (shaped `[batch_size, sequence_len, embed_dim]`).
func Conv1DGraph(ctx *context.Context, embed, mask *Node) *Node {
    g := embed.Graph()
    // 1D Convolution:
    {
        ctx := ctx.In("conv1")
        embed = Activation(embed)
        embed = layers.Dropout(ctx, embed, ConstAsDType(g, DType, *flagDropoutRate))
        embed = layers.Convolution(ctx, embed).ChannelsAfter().KernelSize(7).Filters(*flagTokenEmbeddingSize).Strides(3).Done()
        AssertNoError(g.Error())
        embed = Normalize(ctx, embed)
    }
    {
        ctx := ctx.In("conv2")
        embed = Activation(embed)
        embed = layers.Convolution(ctx, embed).ChannelsAfter().KernelSize(7).Filters(*flagTokenEmbeddingSize).Strides(3).Done()
        AssertNoError(g.Error())
        embed = Normalize(ctx, embed)
    }
    return embed
}

%% --model=cnn --steps=20000 --optimizer=adamw
trainModel()

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Platform: CUDA


Training (20000 steps):  100% [========================================] (461 steps/s) [loss=0.085] [~loss=0.095] [~acc=96.84%]        


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.011800e+01 L3.461121e+01,2.006078e+01 L5.158486e+01,2.099553e+01 L6.855851e+01,2.910314e+01 L8.553216e+01,2.006078e+01 L1.025058e+02,2.280782e+01 L1.194795e+02,5.024018e+01 L1.364531e+02,6.517725e+01 L1.534268e+02,9.623417e+01 L1.704004e+02,1.354941e+02 L1.873741e+02,1.432011e+02 L2.043477e+02,1.507173e+02 L2.213214e+02,1.605228e+02 L2.382950e+02,1.542847e+02 L2.552686e+02,1.637849e+02 L2.722423e+02,1.681726e+02 L2.892159e+02,1.684587e+02 L3.061896e+02,1.690882e+02 L3.231632e+02,1.691264e+02 L3.401369e+02,1.698894e+02 L3.571105e+02,1.711485e+02 L3.740842e+02,1.684969e+02 L3.910578e+02,1.624304e+02 L4.080315e+02,1.745632e+02 L4.250051e+02,1.760322e+02 L4.419788e+02,1.713774e+02 L4.589524e+02,1.748303e+02 L4.759261e+02,1.756125e+02 L4.928997e+02,1.709577e+02 L5.098734e+02,1.755362e+02 L5.268470e+02,1.749448e+02 L5.438207e+02,1.736666e+02 L5.607943e+02,1.724076e+02 L5.777680e+02,1.642237e+02 L5.947416e+02,1.747349e+02 L6.117153e+02,1.733233e+02 L6.286889e+02,1.736476e+02 L6.456626e+02,1.731706e+02 L6.626362e+02,1.763374e+02 L6.796099e+02,1.765663e+02 L6.965835e+02,1.723885e+02 L7.135572e+02,1.709196e+02 L7.305308e+02,1.744679e+02 L7.475045e+02,1.685922e+02 L7.644781e+02,1.696224e+02 L7.814518e+02,1.744297e+02 L7.984254e+02,1.765472e+02 L8.153991e+02,1.645480e+02 L8.323727e+02,1.757651e+02 L8.493464e+02,1.701947e+02 L8.663200e+02,1.705380e+02 "/> <path d="M1.768000e+01,2.007986e+01 L3.461121e+01,2.011800e+01 L5.158486e+01,2.368535e+01 L6.855851e+01,3.997688e+01 L8.553216e+01,2.019433e+01 L1.025058e+02,2.410505e+01 L1.194795e+02,7.643252e+01 L1.364531e+02,8.385337e+01 L1.534268e+02,1.258604e+02 L1.704004e+02,1.674095e+02 L1.873741e+02,1.819841e+02 L2.043477e+02,1.916942e+02 L2.213214e+02,2.012516e+02 L2.382950e+02,1.980658e+02 L2.552686e+02,2.105610e+02 L2.722423e+02,2.172379e+02 L2.892159e+02,2.192028e+02 L3.061896e+02,2.212631e+02 L3.231632e+02,2.224840e+02 L3.401369e+02,2.248877e+02 L3.571105e+02,2.282070e+02 L3.740842e+02,2.274249e+02 L3.910578e+02,2.204428e+02 L4.080315e+02,2.361429e+02 L4.250051e+02,2.380125e+02 L4.419788e+02,2.357423e+02 L4.589524e+02,2.405497e+02 L4.759261e+02,2.425146e+02 L4.928997e+02,2.386611e+02 L5.098734e+02,2.442506e+02 L5.268470e+02,2.454333e+02 L5.438207e+02,2.449182e+02 L5.607943e+02,2.453952e+02 L5.777680e+02,2.374211e+02 L5.947416e+02,2.484284e+02 L6.117153e+02,2.483711e+02 L6.286889e+02,2.497828e+02 L6.456626e+02,2.500499e+02 L6.626362e+02,2.516523e+02 L6.796099e+02,2.521292e+02 L6.965835e+02,2.505840e+02 L7.135572e+02,2.506413e+02 L7.305308e+02,2.524917e+02 L7.475045e+02,2.493631e+02 L7.644781e+02,2.502788e+02 L7.814518e+02,2.539797e+02 L7.984254e+02,2.543803e+02 L8.153991e+02,2.484665e+02 L8.323727e+02,2.548000e+02 L8.493464e+02,2.535028e+02 L8.663200e+02,2.536172e+02 " vector-effect="non-scaling-stroke"/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.200000e+00 L3.461121e+01,2.724228e+01 L5.158486e+01,2.303667e+01 L6.855851e+01,1.843693e+01 L8.553216e+01,2.026389e+01 L1.025058e+02,2.826392e+01 L1.194795e+02,2.849970e+01 L1.364531e+02,5.308761e+01 L1.534268e+02,1.088253e+02 L1.704004e+02,1.364523e+02 L1.873741e+02,1.548724e+02 L2.043477e+02,1.709578e+02 L2.213214e+02,1.805582e+02 L2.382950e+02,1.848202e+02 L2.552686e+02,1.904003e+02 L2.722423e+02,1.944073e+02 L2.892159e+02,2.005617e+02 L3.061896e+02,1.977869e+02 L3.231632e+02,2.077879e+02 L3.401369e+02,2.086590e+02 L3.571105e+02,2.099818e+02 L3.740842e+02,2.125433e+02 L3.910578e+02,2.175634e+02 L4.080315e+02,2.126551e+02 L4.250051e+02,2.185679e+02 L4.419788e+02,2.179140e+02 L4.589524e+02,2.203257e+02 L4.759261e+02,2.233215e+02 L4.928997e+02,2.260679e+02 L5.098734e+02,2.258657e+02 L5.268470e+02,2.285906e+02 L5.438207e+02,2.256114e+02 L5.607943e+02,2.309598e+02 L5.777680e+02,2.286230e+02 L5.947416e+02,2.312750e+02 L6.117153e+02,2.311705e+02 L6.286889e+02,2.314447e+02 L6.456626e+02,2.330832e+02 L6.626362e+02,2.351043e+02 L6.796099e+02,2.355631e+02 L6.965835e+02,2.347531e+02


Results on train-eval:


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.494666e+02 L3.461121e+01,2.470866e+02 L5.158486e+01,2.440103e+02 L6.855851e+01,2.438453e+02 L8.553216e+01,2.440388e+02 L1.025058e+02,2.437163e+02 L1.194795e+02,2.429047e+02 L1.364531e+02,2.358334e+02 L1.534268e+02,2.218264e+02 L1.704004e+02,1.895872e+02 L1.873741e+02,1.832833e+02 L2.043477e+02,1.771548e+02 L2.213214e+02,1.656264e+02 L2.382950e+02,1.829458e+02 L2.552686e+02,1.633205e+02 L2.722423e+02,1.590012e+02 L2.892159e+02,1.602683e+02 L3.061896e+02,1.586635e+02 L3.231632e+02,1.675297e+02 L3.401369e+02,1.660547e+02 L3.571105e+02,1.641729e+02 L3.740842e+02,1.712483e+02 L3.910578e+02,1.953207e+02 L4.080315e+02,1.560244e+02 L4.250051e+02,1.598726e+02 L4.419788e+02,1.726502e+02 L4.589524e+02,1.671295e+02 L4.759261e+02,1.677907e+02 L4.928997e+02,1.844051e+02 L5.098734e+02,1.700779e+02 L5.268470e+02,1.722311e+02 L5.438207e+02,1.757676e+02 L5.607943e+02,1.939691e+02 L5.777680e+02,2.155657e+02 L5.947416e+02,1.873686e+02 L6.117153e+02,1.848468e+02 L6.286889e+02,1.870599e+02 L6.456626e+02,1.923739e+02 L6.626362e+02,1.900778e+02 L6.796099e+02,1.856261e+02 L6.965835e+02,2.034714e+02 L7.135572e+02,2.090451e+02 L7.305308e+02,2.029836e+02 L7.475045e+02,2.183409e+02 L7.644781e+02,2.261047e+02 L7.814518e+02,2.041946e+02 L7.984254e+02,1.973789e+02 L8.153991e+02,2.548000e+02 L8.323727e+02,1.969629e+02 L8.493464e+02,2.293023e+02 L8.663200e+02,2.261958e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.496178e+02 L3.461121e+01,2.469387e+02 L5.158486e+01,2.438790e+02 L6.855851e+01,2.436342e+02 L8.553216e+01,2.437147e+02 L1.025058e+02,2.432637e+02 L1.194795e+02,2.419096e+02 L1.364531e+02,2.306655e+02 L1.534268e+02,1.952250e+02 L1.704004e+02,1.540683e+02 L1.873741e+02,1.337481e+02 L2.043477e+02,1.205123e+02 L2.213214e+02,1.063001e+02 L2.382950e+02,1.103934e+02 L2.552686e+02,9.206557e+01 L2.722423e+02,8.228090e+01 L2.892159e+02,7.815116e+01 L3.061896e+02,7.494832e+01 L3.231632e+02,7.293414e+01 L3.401369e+02,6.850103e+01 L3.571105e+02,6.331104e+01 L3.740842e+02,6.366948e+01 L3.910578e+02,7.505992e+01 L4.080315e+02,5.064982e+01 L4.250051e+02,4.694809e+01 L4.419788e+02,4.950497e+01 L4.589524e+02,4.195150e+01 L4.759261e+02,3.837211e+01 L4.928997e+02,4.362888e+01 L5.098734e+02,3.504479e+01 L5.268470e+02,3.334511e+01 L5.438207e+02,3.338212e+01 L5.607943e+02,3.194629e+01 L5.777680e+02,4.386774e+01 L5.947416e+02,2.591403e+01 L6.117153e+02,2.672184e+01 L6.286889e+02,2.408881e+01 L6.456626e+02,2.328816e+01 L6.626362e+02,1.935350e+01 L6.796099e+02,1.860584e+01 L6.965835e+02,2.117557e+01 L7.135572e+02,2.094654e+01 L7.305308e+02,1.721395e+01 L7.475045e+02,2.323954e+01 L7.644781e+02,2.041266e+01 L7.814518e+02,1.424598e+01 L7.984254e+02,1.308780e+01 L8.153991e+02,2.397611e+01 L8.323727e+02,1.256465e+01 L8.493464e+02,1.459349e+01 L8.663200e+02,1.456794e+01 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.502973e+02 L3.461121e+01,2.395633e+02 L5.158486e+01,2.422632e+02 L6.855851e+01,2.470445e+02 L8.553216e+01,2.435411e+02 L1.025058e+02,2.398745e+02 L1.194795e+02,2.462290e+02 L1.364531e+02,2.396286e+02 L1.534268e+02,2.264111e+02 L1.704004e+02,1.527183e+02 L1.873741e+02,1.519939e+02 L2.043477e+02,1.000025e+02 L2.213214e+02,1.041206e+02 L2.382950e+02,1.896802e+02 L2.552686e+02,1.501695e+02 L2.722423e+02,8.985004e+01 L2.892159e+02,1.060147e+02 L3.061896e+02,1.302393e+02 L3.231632e+02,8.205105e+01 L3.401369e+02,7.187207e+01 L3.571105e+02,7.116645e+01 L3.740842e+02,8.590687e+01 L3.910578e+02,2.916411e+01 L4.080315e+02,8.208030e+01 L4.250051e+02,5.678016e+01 L4.419788e+02,8.191981e+01 L4.589524e+02,7.222045e+01 L4.759261e+02,6.064449e+01 L4.928997e+02,8.766622e+01 L5.098734e+02,7.627867e+01 L5.268470e+02,6.827964e+01 L5.438207e+02,3.709332e+01 L5.607943e+02,4.020911e+01 L5.777680e+02,1.600236e+02 L5.947416e+02,4.426914e+01 L6.117153e+02,2.936928e+01 L6.286889e+02,2.203271e+01 L6.456626e+02,6.790196e+01 L6.626362e+02,8.287797e+01 L6.796099e+02,5.368493e+01 L6.965835e+02,5.977852e+01

	Mean Loss (#loss): 0.040
	Mean Accuracy (#acc): 98.98%
Results on test-eval:
	Mean Loss (#loss): 0.642
	Mean Accuracy (#acc): 81.56%


### Transformer Model

Finally a Transformer version of the model. 

Notice it's not significantly better than our previous simple Bag-Of-Words model. Likely because there is not enough data for the transformer to make any difference. The success of transformers in large-language-models is in large part due to the training with huge amounts of unsupervised (or self-supervised) data.


In [11]:
var (
	flagMaxAttLen           = flag.Int("max_att_len", 200, "Maximum attention length: input will be split in ranges of this size.")
	flagNumAttHeads         = flag.Int("att_heads", 2, "Number of attention heads, if --model=transformer.")
	flagNumAttLayers        = flag.Int("att_layers", 1, "Number of stacked attention layers, if --model=transformer.")
	flagAttKeyQueryEmbedDim = flag.Int("att_key_dim", 8, "Dimension of the Key/Query attention embedding.")    
)

// TransformerGraph is the part of the model that takes the word/token embeddings to a tranformed
// embedding through attention ready to be pooled and read out.
func TransformerGraph(ctx *context.Context, input, embed, mask *Node) *Node {
    var newEmbed *Node
    if *flagMaxAttLen >= *flagMaxLen {
        // One transformer window covers the whole length, which makes it trivial.
        newEmbed = TransformerLayers(ctx.In("transformer"), embed, mask)
        embed = Add(embed, newEmbed)
        return embed
    }

    // Split embedding in multiple split embeddings and apply transformer in each of them.
    attLen := *flagMaxAttLen
    sequenceFrom := 0
    for {
        // x.shape = [batchSize, sequence, embedding]
        sequenceTo := sequenceFrom + attLen
        if sequenceTo > *flagMaxLen {
            sequenceTo = *flagMaxLen
            sequenceFrom = sequenceTo - attLen
        }
        // part = x[:, sequenceFrom:sequenceTo, :]
        residual := Slice(embed, AxisRange(), AxisRange(sequenceFrom, sequenceTo), AxisRange())
        partMask := Slice(mask, AxisRange(), AxisRange(sequenceFrom, sequenceTo))
        // Reuse "transformer" scope.
        part := TransformerLayers(ctx.In("transformer").Checked(false), residual, partMask)
        part = Add(residual, part)
        if newEmbed == nil {
            newEmbed = part
        } else {
            newEmbed = Add(newEmbed, part)
        }

        if sequenceTo == *flagMaxLen {
            // Reached end of parts.
            break
        }
        sequenceFrom += attLen - 20 // Attention window overlap 10 tokens among themselves.
    }
    embed = newEmbed // Notice shape changed to `[batchSize, maxAttLen, embedDim]`
    return embed
}

// TransformerLayers builds the stacked transformer layers for the model.
func TransformerLayers(ctx *context.Context, embed, mask *Node) *Node {
    g := embed.Graph()
    shape := embed.Shape()
    embedDim := shape.Dimensions[2]

    var dropoutRate *Node
    if *flagDropoutRate > 0 {
        dropoutRate = ConstAsDType(g, DType, *flagDropoutRate)
    }

    // Create positional embedding variable: it is 1 in every axis, but for the
    // sequence dimension -- there will be one embedding per position.
    // Shape: [1, maxLen, embedDim]
    posEmbedShape := shape.Copy()
    posEmbedShape.Dimensions[0] = 1
    posEmbedVar := ctx.VariableWithShape("positional", posEmbedShape)
    posEmbed := posEmbedVar.ValueGraph(g)
    embed = Add(embed, posEmbed) // Just add the embeddings, seems to work well.

    // Add the requested number of attention layers.
    for ii := 0; ii < *flagNumAttLayers; ii++ {
        // Each layer in its own scope.
        ctx := ctx.In(fmt.Sprintf("AttLayer_%d", ii))
        residual := embed
        embed = layers.MultiHeadAttention(ctx, embed, embed, embed, *flagNumAttHeads, *flagAttKeyQueryEmbedDim).
            SetKeyMask(mask).SetQueryMask(mask).
            SetOutputDim(embedDim).
            SetValueHeadDim(embedDim).Done()
        if *flagDropoutRate > 0 {
            embed = layers.Dropout(ctx.In("dropout_1"), embed, dropoutRate)
        }
        embed = Normalize(ctx.In("normalization_1"), embed)
        attentionOutput := embed

        // Transformers recipe: 2 dense layers after attention.
        embed = layers.Dense(ctx.In("ffn_1"), embed, true, embedDim)
        embed = Tanh(embed)
        embed = layers.Dense(ctx.In("ffn_2"), embed, true, embedDim)
        if *flagDropoutRate > 0 {
            embed = layers.Dropout(ctx.In("dropout_1"), embed, dropoutRate)
        }
        embed = Add(embed, attentionOutput)
        embed = Normalize(ctx.In("normalization_2"), embed)

        // Residual connection: not part of the usual transfomer layer ...
        if ii > 0 {
            embed = Add(residual, embed)
        }
    }
    return embed
}

%% --model=transformer --steps=30000 --optimizer=adamw --max_len=1000 --max_att_len=200 --dropout=0.2 --max_vocab=10000
trainModel()

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Platform: CUDA


Training (30000 steps):  100% [========================================] (23 steps/s) [loss=0.023] [~loss=0.087] [~acc=98.27%]            


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.200000e+00 L3.462508e+01,5.241065e+00 L5.159844e+01,3.483045e+01 L6.857181e+01,1.753441e+02 L8.554517e+01,1.860218e+02 L1.025185e+02,1.843175e+02 L1.194919e+02,1.815864e+02 L1.364653e+02,1.872333e+02 L1.534386e+02,1.939890e+02 L1.704120e+02,1.876850e+02 L1.873854e+02,1.935167e+02 L2.043587e+02,1.893894e+02 L2.213321e+02,1.910526e+02 L2.383055e+02,1.904160e+02 L2.552788e+02,1.909705e+02 L2.722522e+02,1.904982e+02 L2.892256e+02,1.908473e+02 L3.061989e+02,1.873359e+02 L3.231723e+02,1.900259e+02 L3.401457e+02,1.882600e+02 L3.571190e+02,1.898616e+02 L3.740924e+02,1.886912e+02 L3.910658e+02,1.875618e+02 L4.080391e+02,1.861244e+02 L4.250125e+02,1.871101e+02 L4.419859e+02,1.869253e+02 L4.589592e+02,1.857959e+02 L4.759326e+02,1.863709e+02 L4.929060e+02,1.857138e+02 L5.098793e+02,1.878493e+02 L5.268527e+02,1.866583e+02 L5.438261e+02,1.853647e+02 L5.607994e+02,1.844407e+02 L5.777728e+02,1.860628e+02 L5.947461e+02,1.858575e+02 L6.117195e+02,1.846665e+02 L6.286929e+02,1.849745e+02 L6.456662e+02,1.837220e+02 L6.626396e+02,1.842559e+02 L6.796130e+02,1.835783e+02 L6.965863e+02,1.830444e+02 L7.135597e+02,1.837836e+02 L7.305331e+02,1.831265e+02 L7.475064e+02,1.821819e+02 L7.644798e+02,1.835783e+02 L7.814532e+02,1.833524e+02 L7.984265e+02,1.818329e+02 L8.153999e+02,1.834756e+02 L8.323733e+02,1.819355e+02 L8.493466e+02,1.821614e+02 L8.663200e+02,1.832908e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.241065e+00 L3.462508e+01,5.241065e+00 L5.159844e+01,4.090850e+01 L6.857181e+01,1.932087e+02 L8.554517e+01,2.071923e+02 L1.025185e+02,2.088145e+02 L1.194919e+02,2.054469e+02 L1.364653e+02,2.159192e+02 L1.534386e+02,2.260835e+02 L1.704120e+02,2.205188e+02 L1.873854e+02,2.330650e+02 L2.043587e+02,2.306215e+02 L2.213321e+02,2.305188e+02 L2.383055e+02,2.352211e+02 L2.552788e+02,2.371718e+02 L2.722522e+02,2.409295e+02 L2.892256e+02,2.442560e+02 L3.061989e+02,2.325516e+02 L3.231723e+02,2.444408e+02 L3.401457e+02,2.403135e+02 L3.571190e+02,2.464326e+02 L3.740924e+02,2.454675e+02 L3.910658e+02,2.447899e+02 L4.080391e+02,2.415661e+02 L4.250125e+02,2.483833e+02 L4.419859e+02,2.508474e+02 L4.589592e+02,2.485476e+02 L4.759326e+02,2.492047e+02 L4.929060e+02,2.498002e+02 L5.098793e+02,2.504162e+02 L5.268527e+02,2.515661e+02 L5.438261e+02,2.512581e+02 L5.607994e+02,2.479316e+02 L5.777728e+02,2.513402e+02 L5.947461e+02,2.526338e+02 L6.117195e+02,2.523875e+02 L6.286929e+02,2.527981e+02 L6.456662e+02,2.518536e+02 L6.626396e+02,2.525517e+02 L6.796130e+02,2.512376e+02 L6.965863e+02,2.517920e+02 L7.135597e+02,2.526544e+02 L7.305331e+02,2.508269e+02 L7.475064e+02,2.446256e+02 L7.644798e+02,2.532704e+02 L7.814532e+02,2.533731e+02 L7.984265e+02,2.494716e+02 L8.153999e+02,2.528597e+02 L8.323733e+02,2.484860e+02 L8.493466e+02,2.519562e+02 L8.663200e+02,2.529008e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,6.938906e+01 L3.462508e+01,1.491894e+01 L5.159844e+01,1.047344e+01 L6.857181e+01,1.726257e+02 L8.554517e+01,2.007656e+02 L1.025185e+02,2.088351e+02 L1.194919e+02,2.154236e+02 L1.364653e+02,2.193154e+02 L1.534386e+02,2.214969e+02 L1.704120e+02,2.229109e+02 L1.873854e+02,2.302505e+02 L2.043587e+02,2.292012e+02 L2.213321e+02,2.340020e+02 L2.383055e+02,2.324504e+02 L2.552788e+02,2.355357e+02 L2.722522e+02,2.399573e+02 L2.892256e+02,2.387097e+02 L3.061989e+02,2.415706e+02 L3.231723e+02,2.410529e+02 L3.401457e+02,2.445408e+02 L3.571190e+02,2.461541e+02 L3.740924e+02,2.481752e+02 L3.910658e+02,2.456847e+02 L4.080391e+02,2.473223e+02 L4.250125e+02,2.500164e+02 L4.419859e+02,2.501026e+02 L4.589592e+02,2.469712e+02 L4.759326e+02,2.498376e+02 L4.929060e+02,2.500460e+02 L5.098793e+02,2.505035e+02 L5.268527e+02,2.501575e+02 L5.438261e+02,2.504089e+02 L5.607994e+02,2.500496e+02 L5.777728e+02,2.504306e+02 L5.947461e+02,2.516071e+02 L6.117195e+02,2.514852e+02 L6.286929e+02,2.524340e+02 L6.456662e+02,2.513150e+02 L6.626396e+02,2.516686e+02 L6.796130e+02,2.502891e+02 L6.965863e+02,2.512679e+02


Results on train-eval:


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.546691e+02 L3.462508e+01,2.509594e+02 L5.159844e+01,2.490550e+02 L6.857181e+01,1.519335e+02 L8.554517e+01,1.350942e+02 L1.025185e+02,1.370872e+02 L1.194919e+02,1.426035e+02 L1.364653e+02,1.238971e+02 L1.534386e+02,1.221257e+02 L1.704120e+02,1.215279e+02 L1.873854e+02,1.217639e+02 L2.043587e+02,1.201424e+02 L2.213321e+02,1.253255e+02 L2.383055e+02,1.257104e+02 L2.552788e+02,1.302402e+02 L2.722522e+02,1.348380e+02 L2.892256e+02,1.332912e+02 L3.061989e+02,1.385259e+02 L3.231723e+02,1.349604e+02 L3.401457e+02,1.388744e+02 L3.571190e+02,1.475435e+02 L3.740924e+02,1.629102e+02 L3.910658e+02,1.515261e+02 L4.080391e+02,1.551518e+02 L4.250125e+02,1.764419e+02 L4.419859e+02,1.807417e+02 L4.589592e+02,1.628384e+02 L4.759326e+02,1.734808e+02 L4.929060e+02,1.819694e+02 L5.098793e+02,1.754086e+02 L5.268527e+02,1.827150e+02 L5.438261e+02,1.818229e+02 L5.607994e+02,1.741491e+02 L5.777728e+02,1.768389e+02 L5.947461e+02,1.916228e+02 L6.117195e+02,1.876691e+02 L6.286929e+02,1.978711e+02 L6.456662e+02,2.000640e+02 L6.626396e+02,1.955366e+02 L6.796130e+02,1.948869e+02 L6.965863e+02,1.972863e+02 L7.135597e+02,2.106780e+02 L7.305331e+02,2.062032e+02 L7.475064e+02,2.143088e+02 L7.644798e+02,2.102236e+02 L7.814532e+02,2.048415e+02 L7.984265e+02,2.130007e+02 L8.153999e+02,2.061916e+02 L8.323733e+02,2.197878e+02 L8.493466e+02,2.188065e+02 L8.663200e+02,2.068364e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.548000e+02 L3.462508e+01,2.507542e+02 L5.159844e+01,2.483291e+02 L6.857181e+01,1.297273e+02 L8.554517e+01,1.057583e+02 L1.025185e+02,1.038841e+02 L1.194919e+02,1.049168e+02 L1.364653e+02,8.810819e+01 L1.534386e+02,7.903648e+01 L1.704120e+02,7.875669e+01 L1.873854e+02,6.904733e+01 L2.043587e+02,7.253468e+01 L2.213321e+02,6.719601e+01 L2.383055e+02,6.467111e+01 L2.552788e+02,6.053024e+01 L2.722522e+02,5.815182e+01 L2.892256e+02,5.532026e+01 L3.061989e+02,5.825434e+01 L3.231723e+02,5.293361e+01 L3.401457e+02,5.269416e+01 L3.571190e+02,4.619931e+01 L3.740924e+02,4.495543e+01 L3.910658e+02,4.764400e+01 L4.080391e+02,4.849726e+01 L4.250125e+02,4.033200e+01 L4.419859e+02,3.736763e+01 L4.589592e+02,4.270838e+01 L4.759326e+02,4.389778e+01 L4.929060e+02,3.755304e+01 L5.098793e+02,3.759189e+01 L5.268527e+02,3.497593e+01 L5.438261e+02,3.576125e+01 L5.607994e+02,4.279896e+01 L5.777728e+02,3.678888e+01 L5.947461e+02,3.260446e+01 L6.117195e+02,3.590394e+01 L6.286929e+02,3.276669e+01 L6.456662e+02,3.268631e+01 L6.626396e+02,3.386625e+01 L6.796130e+02,3.380376e+01 L6.965863e+02,3.295620e+01 L7.135597e+02,3.058032e+01 L7.305331e+02,3.280060e+01 L7.475064e+02,4.176090e+01 L7.644798e+02,2.960538e+01 L7.814532e+02,3.037213e+01 L7.984265e+02,3.341565e+01 L8.153999e+02,2.971872e+01 L8.323733e+02,3.481089e+01 L8.493466e+02,3.045259e+01 L8.663200e+02,2.960029e+01 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.478347e+02 L3.462508e+01,2.457026e+02 L5.159844e+01,2.504178e+02 L6.857181e+01,1.244030e+02 L8.554517e+01,1.624105e+02 L1.025185e+02,1.138184e+02 L1.194919e+02,8.778619e+01 L1.364653e+02,5.579185e+01 L1.534386e+02,9.045705e+01 L1.704120e+02,9.538129e+01 L1.873854e+02,2.641159e+01 L2.043587e+02,4.798566e+01 L2.213321e+02,5.839820e+01 L2.383055e+02,7.014200e+01 L2.552788e+02,5.277667e+01 L2.722522e+02,1.551337e+01 L2.892256e+02,1.567419e+01 L3.061989e+02,8.332889e+01 L3.231723e+02,4.532461e+01 L3.401457e+02,1.830046e+01 L3.571190e+02,1.125533e+01 L3.740924e+02,8.912841e+00 L3.910658e+02,2.410318e+01 L4.080391e+02,1.342063e+01 L4.250125e+02,8.697318e+00 L4.419859e+02,5.361299e+01 L4.589592e+02,3.945452e+01 L4.759326e+02,1.095591e+02 L4.929060e+02,8.535801e+00 L5.098793e+02,9.038528e+01 L5.268527e+02,9.428448e+00 L5.438261e+02,4.658799e+01 L5.607994e+02,8.310600e+00 L5.777728e+02,8.997530e+00 L5.947461e+02,6.061855e+01 L6.117195e+02,4.149010e+01 L6.286929e+02,6.250611e+01 L6.456662e+02,5.571452e+01 L6.626396e+02,5.865915e+00 L6.796130e+02,5.650797e+01 L6.965863e+02,8.176560e+00

	Mean Loss (#loss): 0.082
	Mean Accuracy (#acc): 98.26%
Results on test-eval:
	Mean Loss (#loss): 0.571
	Mean Accuracy (#acc): 84.70%


In [13]:
%% --model=transformer --steps=3000 --batch=100 --optimizer=adamw --max_len=200 --max_att_len=200 --max_vocab=10000 --att_heads=4 --att_layers=3 --att_key_dim=32
trainModel()

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Platform: CUDA


Training (3000 steps):  100% [========================================] (7 steps/s) [loss=0.134] [~loss=0.159] [~acc=95.15%]          


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,1.941732e+01 L3.437549e+01,1.401598e+01 L5.135394e+01,1.484378e+01 L6.833240e+01,1.484378e+01 L8.531086e+01,1.484378e+01 L1.022893e+02,2.113501e+01 L1.192678e+02,1.370557e+01 L1.362462e+02,5.294291e+01 L1.532247e+02,1.376881e+02 L1.702032e+02,1.649639e+02 L1.871816e+02,1.754562e+02 L2.041601e+02,1.783741e+02 L2.211385e+02,1.676542e+02 L2.381170e+02,1.916602e+02 L2.550955e+02,1.925707e+02 L2.720739e+02,1.943091e+02 L2.890524e+02,1.931295e+02 L3.060308e+02,1.969994e+02 L3.230093e+02,1.971443e+02 L3.399878e+02,1.985515e+02 L3.569662e+02,1.995242e+02 L3.739447e+02,1.932537e+02 L3.909231e+02,1.998760e+02 L4.079016e+02,2.005796e+02 L4.248801e+02,1.999381e+02 L4.418585e+02,1.989447e+02 L4.588370e+02,1.938952e+02 L4.758154e+02,2.008693e+02 L4.927939e+02,2.009315e+02 L5.097724e+02,1.980342e+02 L5.267508e+02,2.003313e+02 L5.437293e+02,2.013867e+02 L5.607077e+02,2.012212e+02 L5.776862e+02,2.005589e+02 L5.946646e+02,2.015937e+02 L6.116431e+02,2.018006e+02 L6.286216e+02,2.009728e+02 L6.456000e+02,2.019041e+02 L6.625785e+02,2.019248e+02 L6.795569e+02,2.007038e+02 L6.965354e+02,1.987999e+02 L7.135139e+02,2.018627e+02 L7.304923e+02,1.977031e+02 L7.474708e+02,1.977651e+02 L7.644492e+02,1.991517e+02 L7.814277e+02,1.981583e+02 L7.984062e+02,2.015109e+02 L8.153846e+02,2.004762e+02 L8.323631e+02,2.006624e+02 L8.493415e+02,2.011591e+02 L8.663200e+02,1.982618e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,1.983122e+01 L3.437549e+01,5.200000e+00 L5.135394e+01,1.484378e+01 L6.833240e+01,1.484378e+01 L8.531086e+01,1.484378e+01 L1.022893e+02,4.770714e+01 L1.192678e+02,4.182979e+01 L1.362462e+02,8.601322e+01 L1.532247e+02,1.597488e+02 L1.702032e+02,1.846653e+02 L1.871816e+02,1.954059e+02 L2.041601e+02,1.939159e+02 L2.211385e+02,1.913083e+02 L2.381170e+02,2.145486e+02 L2.550955e+02,2.184806e+02 L2.720739e+02,2.157696e+02 L2.890524e+02,2.151488e+02 L3.060308e+02,2.249581e+02 L3.230093e+02,2.261791e+02 L3.399878e+02,2.270483e+02 L3.569662e+02,2.292833e+02 L3.739447e+02,2.251237e+02 L3.909231e+02,2.300490e+02 L4.079016e+02,2.333809e+02 L4.248801e+02,2.329256e+02 L4.418585e+02,2.349951e+02 L4.588370e+02,2.310217e+02 L4.758154e+02,2.385339e+02 L4.927939e+02,2.379544e+02 L5.097724e+02,2.374785e+02 L5.267508e+02,2.372508e+02 L5.437293e+02,2.422797e+02 L5.607077e+02,2.431695e+02 L5.776862e+02,2.400239e+02 L5.946646e+02,2.445147e+02 L6.116431e+02,2.455288e+02 L6.286216e+02,2.450321e+02 L6.456000e+02,2.461082e+02 L6.625785e+02,2.472878e+02 L6.795569e+02,2.479914e+02 L6.965354e+02,2.468739e+02 L7.135139e+02,2.502679e+02 L7.304923e+02,2.468532e+02 L7.474708e+02,2.470395e+02 L7.644492e+02,2.493573e+02 L7.814277e+02,2.463772e+02 L7.984062e+02,2.527926e+02 L8.153846e+02,2.537653e+02 L8.323631e+02,2.546138e+02 L8.493415e+02,2.548000e+02 L8.663200e+02,2.507024e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,3.036487e+01 L3.437549e+01,3.183071e+01 L5.135394e+01,2.546625e+01 L6.833240e+01,2.473845e+01 L8.531086e+01,2.367547e+01 L1.022893e+02,2.431936e+01 L1.192678e+02,2.464683e+01 L1.362462e+02,3.218861e+01 L1.532247e+02,5.724128e+01 L1.702032e+02,1.041301e+02 L1.871816e+02,1.388510e+02 L2.041601e+02,1.646608e+02 L2.211385e+02,1.813729e+02 L2.381170e+02,1.906740e+02 L2.550955e+02,1.994780e+02 L2.720739e+02,2.022661e+02 L2.890524e+02,2.087750e+02 L3.060308e+02,2.117509e+02 L3.230093e+02,2.161072e+02 L3.399878e+02,2.165965e+02 L3.569662e+02,2.180662e+02 L3.739447e+02,2.206107e+02 L3.909231e+02,2.224250e+02 L4.079016e+02,2.249325e+02 L4.248801e+02,2.262720e+02 L4.418585e+02,2.247316e+02 L4.588370e+02,2.276409e+02 L4.758154e+02,2.305469e+02 L4.927939e+02,2.309713e+02 L5.097724e+02,2.315509e+02 L5.267508e+02,2.321784e+02 L5.437293e+02,2.343414e+02 L5.607077e+02,2.358705e+02 L5.776862e+02,2.346040e+02 L5.946646e+02,2.377157e+02 L6.116431e+02,2.390858e+02 L6.286216e+02,2.399727e+02 L6.456000e+02,2.400763e+02 L6.625785e+02,2.377405e+02 L6.795569e+02,2.415287e+02 L6.965354e+02,2.436913e+02

<path d="M1.768000e+01,2.525075e+02 L3.437549e+01,2.525385e+02 L5.135394e+01,2.545864e+02 L6.833240e+01,2.548000e+02 L8.531086e+01,2.529626e+02 L1.022893e+02,2.523414e+02 L1.192678e+02,2.526492e+02 L1.362462e+02,2.502518e+02 L1.532247e+02,2.084959e+02 L1.702032e+02,1.635702e+02 L1.871816e+02,1.444163e+02 L2.041601e+02,1.381631e+02 L2.211385e+02,1.469262e+02 L2.381170e+02,1.159018e+02 L2.550955e+02,1.110371e+02 L2.720739e+02,1.110799e+02 L2.890524e+02,1.146647e+02 L3.060308e+02,1.060318e+02 L3.230093e+02,1.049266e+02 L3.399878e+02,9.969507e+01 L3.569662e+02,1.025531e+02 L3.739447e+02,1.101798e+02 L3.909231e+02,1.042641e+02 L4.079016e+02,1.037671e+02 L4.248801e+02,1.064128e+02 L4.418585e+02,1.020589e+02 L4.588370e+02,1.105001e+02 L4.758154e+02,1.017577e+02 L4.927939e+02,1.032145e+02 L5.097724e+02,1.038845e+02 L5.267508e+02,1.069930e+02 L5.437293e+02,1.090477e+02 L5.607077e+02,1.022441e+02 L5.776862e+02,1.044120e+02 L5.946646e+02,1.169058e+02 L6.116431e+02,1.105009e+02 L6.286216e+02,1.126024e+02 L6.456000e+02,1.149531e+02 L6.625785e+02,1.092011e+02 L6.795569e+02,1.111259e+02 L6.965354e+02,1.261159e+02 L7.135139e+02,1.149960e+02 L7.304923e+02,1.284114e+02 L7.474708e+02,1.308043e+02 L7.644492e+02,1.212009e+02 L7.814277e+02,1.256799e+02 L7.984062e+02,1.228600e+02 L8.153846e+02,1.280231e+02 L8.323631e+02,1.280222e+02 L8.493415e+02,1.285333e+02 L8.663200e+02,1.431096e+02 " vector-effect="non-scaling-stroke"/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.527217e+02 L3.437549e+01,2.526603e+02 L5.135394e+01,2.544351e+02 L6.833240e+01,2.541414e+02 L8.531086e+01,2.519029e+02 L1.022893e+02,2.506882e+02 L1.192678e+02,2.509613e+02 L1.362462e+02,2.469892e+02 L1.532247e+02,1.969789e+02 L1.702032e+02,1.422321e+02 L1.871816e+02,1.187834e+02 L2.041601e+02,1.191609e+02 L2.211385e+02,1.110435e+02 L2.381170e+02,8.463703e+01 L2.550955e+02,7.851679e+01 L2.720739e+02,8.086064e+01 L2.890524e+02,8.247368e+01 L3.060308e+02,6.408413e+01 L3.230093e+02,6.109675e+01 L3.399878e+02,6.010005e+01 L3.569662e+02,5.554302e+01 L3.739447e+02,5.775561e+01 L3.909231e+02,5.560259e+01 L4.079016e+02,4.946860e+01 L4.248801e+02,5.217992e+01 L4.418585e+02,4.556523e+01 L4.588370e+02,4.763794e+01 L4.758154e+02,4.058763e+01 L4.927939e+02,3.938613e+01 L5.097724e+02,4.000182e+01 L5.267508e+02,4.265004e+01 L5.437293e+02,3.502068e+01 L5.607077e+02,3.515296e+01 L5.776862e+02,4.063571e+01 L5.946646e+02,2.965593e+01 L6.116431e+02,2.896104e+01 L6.286216e+02,2.720647e+01 L6.456000e+02,2.477151e+01 L6.625785e+02,2.370385e+01 L6.795569e+02,2.353669e+01 L6.965354e+02,2.345084e+01 L7.135139e+02,2.033549e+01 L7.304923e+02,2.235837e+01 L7.474708e+02,2.076047e+01 L7.644492e+02,1.852220e+01 L7.814277e+02,2.066217e+01 L7.984062e+02,1.365467e+01 L8.153846e+02,1.229911e+01 L8.323631e+02,1.130958e+01 L8.493415e+02,1.169968e+01 L8.663200e+02,1.577241e+01 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.480221e+02 L3.437549e+01,2.496108e+02 L5.135394e+01,2.502985e+02 L6.833240e+01,2.526460e+02 L8.531086e+01,2.490620e+02 L1.022893e+02,2.505799e+02 L1.192678e+02,2.503131e+02 L1.362462e+02,2.497770e+02 L1.532247e+02,2.027357e+02 L1.702032e+02,1.385278e+02 L1.871816e+02,1.303090e+02 L2.041601e+02,1.091494e+02 L2.211385e+02,8.732586e+01 L2.381170e+02,1.028203e+02 L2.550955e+02,8.950318e+01 L2.720739e+02,5.140055e+01 L2.890524e+02,6.936852e+01 L3.060308e+02,1.171119e+02 L3.230093e+02,7.315102e+01 L3.399878e+02,6.894750e+01 L3.569662e+02,5.992039e+01 L3.739447e+02,8.549176e+01 L3.909231e+02,9.245955e+01 L4.079016e+02,1.408147e+02 L4.248801e+02,7.261443e+01 L4.418585e+02,9.404018e+01 L4.588370e+02,6.715228e+01 L4.758154e+02,6.204595e+01 L4.927939e+02,1.044877e+02 L5.097724e+02,8.295873e+01 L5.267508e+02,5.388994e+01 L5.437293e+02,4.142290e+01 L5.607077e+02,8.247322e+00 L5.776862e+02,4.612393e+01 L5.946646e+02,5.200000e+00 L6.116431e+02,1.943506e+01 L6.286216e+02,3.738670e+01 L6.456000e+02,4.228367e+01 L6.625785e+02,2.411150e+01 L6.795569e+02,2.616034e+01 L6.965354e+02,2.054285e+01


Results on train-eval:
	Mean Loss (#loss): 0.143
	Mean Accuracy (#acc): 95.59%
Results on test-eval:
	Mean Loss (#loss): 0.439
	Mean Accuracy (#acc): 85.45%
